In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 108.2 MB/s eta 0:00:00


In [ ]:
!pip install torch==2.5.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.4/838.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1

In [ ]:
%cd /content/gdrive/MyDrive/data

/content/gdrive/MyDrive/data


In [ ]:
!curl -fsSL https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
!chmod +x cloudflared

In [ ]:
!pip install streamlit colabcode opencv-python numpy pillow scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.5 MB/s eta 0:00:00
Requested uvicorn==0.13.1 from https://files.pythonhosted.org/packages/ef/67/546c35e9fffb585ea0608ba3bdcafe17ae402e304367203d0b08d6c23051/uvicorn-0.13.1-py3-none-any.whl (from colabcode) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^
Please use pip<24.1 if you need to use this version.
INFO: pip is looking at multiple versions of colabcode to determine which version is compatible with other requirements. This could take a while.
  Using cached uvicorn-0.13.1-py3-none-any.whl.metadata (4.6 kB)
Requested uvicorn==0.13.1 from https://files.pythonhosted.org/packages/ef/67/546c35e9fffb585ea0608ba3bdcafe17ae402e304367203d0b08d6c23051/uvicorn-0.13.1-py3-none-any.whl (from colabcode) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    python-dotenv (>=0.13.*) ; extra == 'standard

In [ ]:
%%writefile app.py
import streamlit as st
from PIL import Image
import os
import cv2
import pandas as pd
import numpy as np
import pywt
from scipy.stats import entropy
from skimage.feature import local_binary_pattern

HOME= r'/content/gdrive/MyDrive/M.ScProject'
#HOME_LBP= r'/content/drive/MyDrive/Fused_Train_LBP'
#HOME_Thermal= r'/content/drive/MyDrive/Thermal_Train'
#HOME_Visual= r'/content/drive/MyDrive/Visual_Train'
BASE_DRIVE_PATH = "/content/gdrive/MyDrive/data"

def apply_sharpen_filter(image):
    # Define the sharpening kernel
    sharpen_kernel = np.array([[-1, -1, -1],
                                [-1, 9, -1],
                                [-1, -1, -1]])
    # Apply the kernel to the image
    sharpened_image = cv2.filter2D(image, -1, sharpen_kernel)
    return sharpened_image

def morphology_hat_transform(image):
    """
    Compute the Morphology-Hat Transform Image (MT) using the formula:
    MT(x, y) = f(x, y) + THT(x, y) + BHT(x, y)

    """

    # Define a structuring element (kernel) for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    print(kernel)
    # Compute the Top-Hat Transform (THT)
    top_hat = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)

    # Compute the Black-Hat Transform (BHT)
    black_hat = cv2.morphologyEx(image, cv2.MORPH_BLACKHAT, kernel)

    # Compute the Morphology-Hat Transform (MT)
    mt_image = cv2.add(image, cv2.add(top_hat, black_hat))
    return mt_image

def high_boost_filter(image):
    # Define the 3×3 high-boost filter kernel
    high_boost_kernel = np.array([[ 0,  -1,  0],
                               [-1,  5.3, -1],
                               [ 0,  -1,  0]], dtype=np.float32)
    # Apply the filter using 2D convolution
    high_boost_image = cv2.filter2D(image, -1, high_boost_kernel)
    # Clip values to stay within 0-255 range and convert to uint8
    high_boost_image = np.clip(high_boost_image, 0, 255).astype(np.uint8)
    return high_boost_image

def mean_value(image):
    return np.mean(image)

def max_value(image):
    return np.max(image)

# To compute weights
def compute_weights(ma, mb):
    total = abs(ma) + abs(mb)
    wa = abs(ma) / total
    wb = abs(mb) / total
    return wa, wb

def dwt_fusion(image1, image2):

    image1 = high_boost_filter(image1)
    #To Apply DWT on both images (Single-level DWT)
    coeffs1 = pywt.dwt2(image1, 'db1')
    coeffs2 = pywt.dwt2(image2, 'db1')

    # Extract approximation(LL) and detail coefficients
    LL1, (LH1, HL1, HH1) = coeffs1
    LL2, (LH2, HL2, HH2) = coeffs2

    # Mean weighted fusion rule for approximation coefficients
    mean_image1 = mean_value(image1)
    mean_image2 = mean_value(image2)
    w1, w2 = compute_weights(mean_image1, mean_image2)

    # Approximation coeffiecient for fused image
    LL = w1 * LL1 + w2 * LL2

    # Max weighted fusion rule for detail coefficients
    max_image1 = max_value(image1)
    max_image2 = max_value(image2)
    w3, w4 = compute_weights(max_image1, max_image2)

    # Detail coefficients to perform fusion
    LH = w3 * LH1 + w4 * LH2
    HL = w3 * HL1 + w4 * HL2
    HH = w3 * HH1 + w4 * HH2

    # Reconstruct the fused image using IDWT
    fused_image = pywt.idwt2((LL, (LH, HL, HH)), 'db1')
    fused_image = np.clip(fused_image, 0, 255).astype(np.uint8)   #np.uint8(fused_image)
    #fused_image_final=cv2.resize(fused_image,(640,512))
    return fused_image

def modal_fusion_dwt(visible_image_path,thermal_image_path,output_folder):
    modified_visible = None
    modified_thermal = None

    #To extractjust the filename from visible image path
    filename = os.path.splitext(os.path.basename(visible_image_path))[0]

    #os.chdir(output_folder)

    visible_image = cv2.imread(visible_image_path)
    thermal_image = cv2.imread(thermal_image_path)

    # Resize the input images
    visible_image = cv2.resize(visible_image, (640, 512))
    thermal_image = cv2.resize(thermal_image, (640, 512))

    # Convert the image to grayscale if it's in color
    if len(visible_image.shape) == 3:
        visible_image = cv2.cvtColor(visible_image, cv2.COLOR_BGR2GRAY)
    # Convert the image to grayscale if it's in color
    if len(thermal_image.shape) == 3:
       thermal_image = cv2.cvtColor(thermal_image, cv2.COLOR_BGR2GRAY)

    modified_visible = morphology_hat_transform(visible_image)
    modified_thermal = apply_sharpen_filter(thermal_image)
    dwt_fused = dwt_fusion(modified_visible, modified_thermal)

    dwt_fused = cv2.resize(dwt_fused, (640, 512))
    dwt_fused = cv2.normalize(dwt_fused, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    filename = "Fused.jpg"
    output_path = os.path.join(output_folder,filename)
    cv2.imwrite(output_path,dwt_fused)
    return output_path


def modal_fusion_lbp(rgb_path, thermal_path, output_folder="output", alpha=0.6, beta=0.4):
    radii = [1, 3, 5]
    num_points = 8
    method = 'uniform'

    def apply_lbp(image_path):
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img, (640, 512))

        if img is None:
            raise ValueError(f"Error loading image: {image_path}")

        if len(img.shape) == 3:
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        else:
            img_gray = img

        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        img_eq = clahe.apply(img_gray)

        img_smooth = cv2.medianBlur(img_eq, 5)

        lbp_combined = np.zeros_like(img_smooth, dtype=np.float32)
        for r in radii:
            lbp = local_binary_pattern(img_smooth, P=num_points, R=r, method=method)
            lbp = cv2.normalize(lbp, None, 0, 255, cv2.NORM_MINMAX)
            lbp_combined += lbp

        lbp_combined = cv2.normalize(lbp_combined, None, 0, 255, cv2.NORM_MINMAX)
        lbp_combined = np.uint8(lbp_combined)

        blurred_lbp = cv2.GaussianBlur(lbp_combined, (5, 5), 0)

        mask = img_eq > 100
        lbp_combined = np.where(mask, lbp_combined, blurred_lbp)

        gamma = 1.2 if np.mean(img_eq) < 128 else 0.9
        img_gamma = np.power(img_eq / 255.0, gamma) * 255
        img_gamma = np.uint8(img_gamma)

        final_output = cv2.addWeighted(img_gamma, 0.85, lbp_combined, 0.15, 0)

        gaussian_blurred = cv2.GaussianBlur(final_output, (3, 3), 1.0)
        final_output = cv2.addWeighted(final_output, 1.5, gaussian_blurred, -0.5, 0)

        return final_output

    lbp_rgb = apply_lbp(rgb_path)
    lbp_thermal = apply_lbp(thermal_path)

    if lbp_rgb.shape != lbp_thermal.shape:
        print(f"Resizing thermal image to match RGB dimensions...")
        lbp_thermal = cv2.resize(lbp_thermal, (lbp_rgb.shape[1], lbp_rgb.shape[0]))

    gray_norm = lbp_rgb.astype(np.float32) / 255.0
    thermal_norm = lbp_thermal.astype(np.float32) / 255.0

    fused_image = cv2.addWeighted(gray_norm, alpha, thermal_norm, beta, 0)

    fused_image = (fused_image * 255).astype(np.uint8)

    os.makedirs(output_folder, exist_ok=True)

    output_path = os.path.join(output_folder, "Fused.jpg")
    cv2.imwrite(output_path, fused_image)
    return output_path

#Information Entropy
def compute_entropy(img):
  # Compute the histogram of the image with 256 bins,
  #covering pixel values from 0 to 255,
  # `density=True` to ensure that the histogram is normalized
  #(i.e., it represents a probability distribution)
  hist, _ = np.histogram(img, bins=256, range=[0, 256], density=True)
  # Compute the entropy of the histogram using base 2 (Shannon entropy)
  return entropy(hist, base=2)

#Mean Gradient
def compute_meangradient(img):
  # Convert to float for precise calculations
  img = img.astype(np.float32)
  # Compute gradients in x and y directions
  gx, gy = np.gradient(img)  # Difference along rows (x-direction)
  # Compute Gradient G using the formula
  G = (gx**2 + gy**2) / 2
  # Compute Mean Gradient (MG)
  MG = np.mean(np.sqrt(G))
  return MG

#Spatial Frequency
def compute_spatialfrequency(img):
  M, N = img.shape  # Image dimensions
  # Compute row frequency (RF)
  RF_squared = np.sum((np.diff(img, axis=1))**2) / (M * N)
  RF = np.sqrt(RF_squared)
  # Compute column frequency (CF)
  CF_squared = np.sum((np.diff(img, axis=0))**2) / (M * N)
  CF = np.sqrt(CF_squared)
  # Compute spatial frequency (SF)
  SF = np.sqrt(RF**2 + CF**2)
  return SF

def compute_metrics(visible_image_path, thermal_image_path, fused_image_path):
  visible_image = cv2.imread(visible_image_path)
  thermal_image = cv2.imread(thermal_image_path)
  fused_image = cv2.imread(fused_image_path)

  # Resize the input images
  visible_image = cv2.resize(visible_image, (640, 512))
  thermal_image = cv2.resize(thermal_image, (640, 512))
  fused_image = cv2.resize(fused_image, (640, 512))
  # Convert the image to grayscale if it's in color
  visible_image = cv2.cvtColor(visible_image, cv2.COLOR_BGR2GRAY)
  thermal_image = cv2.cvtColor(thermal_image, cv2.COLOR_BGR2GRAY)
  fused_image = cv2.cvtColor(fused_image, cv2.COLOR_BGR2GRAY)

  #To Create a dataframe containing Information Entropy,Mean Gradient and Spatial Frequency for
  #Visible, Thermal and Fused Images
  metrics = {
        "Metric": ["Information Entropy", "Mean Gradient", "Spatial Frequency"],
        "Visible Image": [
            compute_entropy(visible_image),
            compute_meangradient(visible_image),
            compute_spatialfrequency(visible_image),
        ],
        "Thermal Image": [
            compute_entropy(thermal_image),
            compute_meangradient(thermal_image),
            compute_spatialfrequency(thermal_image),
        ],
        "Fused Image": [
            compute_entropy(fused_image),
            compute_meangradient(fused_image),
            compute_spatialfrequency(fused_image),
        ],
    }
  return pd.DataFrame(metrics)

def pred(img, model):
    # Run detection on the provided image.
    if st.session_state.fusion_method == "DWT":
      cmd = f'python /content/gdrive/MyDrive/M.ScProject/yolov9/detect.py --weights /content/gdrive/MyDrive/M.ScProject/yolov9/runs/train/exp3/weights/best_dwt.pt --conf 0.6 --source {img} --device 0 --exist-ok --line-thickness 2'
    elif st.session_state.fusion_method == "LBP":
      cmd = f'python /content/gdrive/MyDrive/M.ScProject/yolov9/detect.py --weights  /content/gdrive/MyDrive/M.ScProject/yolov9/runs/train/exp3/weights/best_lbp.pt --conf 0.6 --source {img} --device 0 --exist-ok --line-thickness 2'
    elif st.session_state.fusion_method== "Thermal (no fusion)":
      cmd= f'python /content/gdrive/MyDrive/M.ScProject/yolov9/detect.py --weights  /content/gdrive/MyDrive/M.ScProject/yolov9/runs/train/exp3/weights/best_thermal.pt --conf 0.6 --source {img} --device 0 --exist-ok --line-thickness 2'
    elif st.session_state.fusion_method== "Visual (no fusion)":
      cmd= f'python /content/gdrive/MyDrive/M.ScProject/yolov9/detect.py --weights  /content/gdrive/MyDrive/M.ScProject/yolov9/runs/train/exp3/weights/best_vis.pt --conf 0.6 --source {img} --device 0 --exist-ok --line-thickness 2'
    os.system(cmd)


# st.markdown(
#     """
#     <style>
#     body, .stApp {
#         background-color: white !important;
#         color: black !important;
#     }
#     h1, h2, h3, h4, h5, h6, p, span, div {
#         color: black !important;
#     }
#     div.stButton > button {
#         background-color: #FF4500 !important;
#         color: white !important;
#         border-radius: 10px !important;
#         border: none !important;
#         padding: 10px 20px !important;
#         font-size: 16px !important;
#         font-weight: bold !important;
#     }
#     div.stButton > button:hover {
#         background-color: #E63900 !important; /* Darker shade on hover */
#     }
#     </style>
#     """,
#     unsafe_allow_html=True
# )



st.title("Human Detection with Multi Modal Fusion")

# Initialize session state variables
if "uploaded" not in st.session_state:
    st.session_state.uploaded = False
if "fused" not in st.session_state:
    st.session_state.fused = False
if "fusion_path" not in st.session_state:
    st.session_state.fusion_path = ""
if "fusion_method" not in st.session_state:
    st.session_state.fusion_method = "DWT"  # Default fusion method

st.header("1. Upload Images")

# Upload the Visual and Thermal images.
visual_file = st.file_uploader("Upload Visual Image", type=["jpg", "jpeg", "png"])
thermal_file = st.file_uploader("Upload Thermal Image", type=["jpg", "jpeg", "png"])

# Confirm Upload: Save the uploaded files to Google Drive.
if st.button("Confirm Upload"):
    if visual_file is not None and thermal_file is not None:
        if not os.path.exists(BASE_DRIVE_PATH):
            os.makedirs(BASE_DRIVE_PATH)

        visual_path = os.path.join(BASE_DRIVE_PATH, "Visual.jpg")
        thermal_path = os.path.join(BASE_DRIVE_PATH, "Thermal.jpg")

        with open(visual_path, "wb") as f:
            f.write(visual_file.getbuffer())
        with open(thermal_path, "wb") as f:
            f.write(thermal_file.getbuffer())

        st.success("Images have been uploaded and saved to Google Drive.")
        st.session_state.uploaded = True
        st.session_state.visual_path = visual_path
        st.session_state.thermal_path = thermal_path
    else:
        st.error("Please upload both Visual and Thermal images before confirming.")

# Display the uploaded images.
if st.session_state.uploaded:
    st.subheader("Uploaded Images")
    col1, col2 = st.columns(2)
    with col1:
        st.image(Image.open(st.session_state.visual_path), caption="Visual Image")
    with col2:
        st.image(Image.open(st.session_state.thermal_path), caption="Thermal Image")

st.header("2. Fuse Images")

# Dropdown to select fusion technique.
fusion_method = st.selectbox("Select Fusion Technique", ["DWT", "LBP","Thermal (no fusion)","Visual (no fusion)"], index=0)
st.session_state.fusion_method = fusion_method

# Fusion button.
if st.session_state.uploaded:
    if st.button("Continue"):
        with st.spinner("Fusing images..."):
            if st.session_state.fusion_method == "DWT":
                modal_fusion_dwt(st.session_state.visual_path, st.session_state.thermal_path, BASE_DRIVE_PATH)
                #compute_metrics(st.session_state.visual_path, st.session_state.thermal_path, fused_path)
                st.session_state.fused = True
                st.success(f"Fusion complete using {st.session_state.fusion_method}. Fused image saved as 'Fused.jpg'.")
            elif st.session_state.fusion_method == "LBP":
                modal_fusion_lbp(st.session_state.visual_path, st.session_state.thermal_path, BASE_DRIVE_PATH)
                #compute_metrics(st.session_state.visual_path, st.session_state.thermal_path, fused_path)
                st.session_state.fused = True
                st.success(f"Fusion complete using {st.session_state.fusion_method}. Fused image saved as 'Fused.jpg'.")
            elif st.session_state.fusion_method== "Thermal (no fusion)":
                output_path = os.path.join(BASE_DRIVE_PATH, "Fused.jpg")
                thermal_img = cv2.imread(st.session_state.thermal_path, cv2.IMREAD_UNCHANGED)
                thermal_img = cv2.resize(thermal_img, (640, 512))
                cv2.imwrite(output_path,thermal_img)
                st.success(f"Continuing with Thermal image only")
                st.session_state.fused = False
            elif st.session_state.fusion_method== "Visual (no fusion)":
                output_path = os.path.join(BASE_DRIVE_PATH, "Fused.jpg")
                visual_img = cv2.imread(st.session_state.visual_path, cv2.IMREAD_UNCHANGED)
                visual_img = cv2.resize(visual_img, (640, 512))
                cv2.imwrite(output_path,visual_img)
                st.success(f"Continuing with Visual image only")
                st.session_state.fused = False

        st.session_state.fusion_path = os.path.join(BASE_DRIVE_PATH, "Fused.jpg")
else:
    st.info("Please upload and confirm images to enable fusion.")

# Display the fused image and the quality metrics comparison table.
if st.session_state.fused and st.session_state.fusion_method!= "Thermal (no fusion)" and st.session_state.fusion_method!= "Visual (no fusion)":
    if os.path.exists(st.session_state.fusion_path):
        st.subheader("Fused Image")
        st.image(Image.open(st.session_state.fusion_path), caption="Fused Image")
        metrics_df = compute_metrics(st.session_state.visual_path, st.session_state.thermal_path, st.session_state.fusion_path)
        st.subheader("Quality Metrics Table")
        st.dataframe(metrics_df)
        # Style the DataFrame
        styled_df = metrics_df.style.set_caption("**IE - Information Entropy, MG - Mean Gradient, SF - Spatial Frequency**") \
    .set_table_styles([{
        'selector': 'caption',
        'props': [('color', 'blue'), ('font-size', '16px'), ('font-weight', 'bold'), ('text-align', 'center')]
    }]) \
    .set_properties(**{
        'text-align': 'center',
        'background-color': '#f0f0f0',
        'border': '1px solid #ddd',
        'font-size': '14px'
    })
    else:
        st.error("Fused image not found. Please ensure the fusion process completed successfully.")

st.header("3. Predict")

# Prediction button.
if st.session_state.fused or st.session_state.fusion_method== "Thermal (no fusion)" or st.session_state.fusion_method== "Visual (no fusion)":
    if st.button("Predict"):
        with st.spinner("Running detection..."):
            pred(st.session_state.fusion_path, st.session_state.fusion_method)
        st.success("Detection complete.")
        if st.session_state.fusion_method == "DWT":
          predicted_img_path = os.path.join(HOME, "yolov9", "runs", "detect", "exp", os.path.basename(st.session_state.fusion_path))
        elif st.session_state.fusion_method == "LBP":
          predicted_img_path = os.path.join(HOME, "yolov9", "runs", "detect", "exp", os.path.basename(st.session_state.fusion_path))
        elif st.session_state.fusion_method== "Thermal (no fusion)":
          predicted_img_path = os.path.join(HOME, "yolov9", "runs", "detect", "exp", os.path.basename(st.session_state.fusion_path))
        elif st.session_state.fusion_method== "Visual (no fusion)":
          predicted_img_path = os.path.join(HOME, "yolov9", "runs", "detect", "exp", os.path.basename(st.session_state.fusion_path))


        if os.path.exists(predicted_img_path):
            st.image(Image.open(predicted_img_path), caption="Predicted Image")
        else:
            st.error("Predicted image not found. Please check if detection ran successfully.")
else:
    st.info("Please complete the fusion step to enable prediction.")



Overwriting app.py


In [ ]:
!streamlit run app.py & ./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-04-20T12:44:21Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-04-20T12:44:21Z INF Requesting new quick Tunnel on trycloudflare.com...



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.152.140:8501

2025-04-20T12:44:27Z INF +--------------------------------------------------------------------------------------------+
2025-04-20T12:44:27Z INF